<a href="https://colab.research.google.com/github/UiinKim/Crawling/blob/main/KoBERT_%ED%99%94%EC%9E%A5%ED%92%88%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.6 MB/s eta 0:00:00


In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert-lm")
model = AutoModelForMaskedLM.from_pretrained("monologg/kobert-lm")

Some weights of the model checkpoint at monologg/kobert-lm were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from google.colab import files
uploaded=files.upload()

Saving V&A.csv to V&A.csv
Saving 닥터지.csv to 닥터지.csv
Saving 메이크프렘.csv to 메이크프렘.csv
Saving 빌리프.csv to 빌리프.csv
Saving 아벤느.csv to 아벤느.csv


In [5]:
import pandas as pd


# Assuming 'files' is a list of pandas DataFrames
files = [pd.read_csv(file_path, encoding='cp949') for file_path in uploaded]

# Assuming 'review_수정' is the second column (index 1)
docs = [' '.join(map(str, file.iloc[:, -1])) for file in files]
stopwords = {'좋아요', '너무', '사용', '피부','제품','구매','좋은','같아요','있어요','쓰고','많이','만족','아주','좋고','좋네요','좋습니다','쓰는','아직','고객님','닥터지'}

preprocessed_docs=[]
for doc in docs:
  words=doc.split()
  filtered_words=[word for word in words if word not in stopwords]
  preprocessed_docs.append(' '.join(filtered_words))


In [13]:
from transformers import pipeline
import torch

# 키워드 추출을 위한 파이프라인 생성
fill_mask = pipeline(
    task="fill-mask",
    model="monologg/kobert-lm",
    tokenizer="monologg/kobert-lm"
)

max_seq_length = tokenizer.model_max_length

# 문장에서 마스킹된 단어의 예측 결과를 가져오기
encoded_inputs = tokenizer(preprocessed_docs, return_tensors='pt', padding=True, truncation=True, max_length=max_seq_length)
predictions = fill_mask(encoded_inputs.input_ids)

# 가장 확률이 높은 예측 결과 가져오기
for i, preprocessed_doc in enumerate(preprocessed_docs):
    top_prediction = predictions[i]['token_str']
    print(f"원래 문장 {i + 1}:", preprocessed_doc)
    print("추출된 키워드:", top_prediction)

Some weights of the model checkpoint at monologg/kobert-lm were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: ignored